Standard imports

In [5]:
# some standard python imports #
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline


Lens Model

In [10]:
# imports #
from lenstronomy.LensModel.lens_model import LensModel
from ipywidgets import interact
from lenstronomy.Plots import lens_plot
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.Data.pixel_grid import PixelGrid
from lenstronomy.ImSim.image_model import ImageModel
import lenstronomy.Util.image_util as image_util
import warnings
warnings.filterwarnings("ignore")

#the source redshift
z_source = 1.5

#creating the list of models to be used #

lens_model_list = ['NIE', 'SHEAR', 'SIS']

#the light models
source_light_model_list = ['SERSIC']
lightModel_source = LightModel(light_model_list=source_light_model_list)
    
lens_light_model_list = ['NIE','SERSIC']
lightModel_lens = LightModel(light_model_list=lens_light_model_list)

#the point source model
point_source_model_list = ['SOURCE_POSITION']


#the data
deltaPix = 0.05  # size of pixel in angular coordinates #

ra_at_xy_0, dec_at_xy_0 = -2.5, -2.5 # coordinate in angles (RA/DEC) at the position of the pixel edge (0,0)
transform_pix2angle = np.array([[1, 0], [0, 1]]) * deltaPix  # linear translation matrix of a shift in pixel in a shift in coordinates
kwargs_pixel = {'nx': 100, 'ny': 100,  # number of pixels per axis
            'ra_at_xy_0': ra_at_xy_0,  # RA at pixel (0,0)
            'dec_at_xy_0': dec_at_xy_0,  # DEC at pixel (0,0)
            'transform_pix2angle': transform_pix2angle} 
pixel_grid = PixelGrid(**kwargs_pixel)
    
from lenstronomy.Data.psf import PSF
kwargs_psf = {'psf_type': 'GAUSSIAN',  # type of PSF model (supports 'GAUSSIAN' and 'PIXEL')
                'fwhm': 0.1,  # full width at half maximum of the Gaussian PSF (in angular units)
                'pixel_size': deltaPix  # angular scale of a pixel (required for a Gaussian PSF to translate the FWHM into a pixel scale)
                 }
psf = PSF(**kwargs_psf)


#numerics for image
kwargs_numerics = {'supersampling_factor': 1, 'supersampling_convolution': False}

#interactive function
def function(theta_E, e1, e2, ext_shear_g1, ext_shear_g2,theta_E_sis,centre_x_sis,centre_y_sis,pert_redshift,amp_source,amp_nie,beta_r,beta_d,PlotImages,PlotLensMap):
    
    #the lens model
    redshift_list = [0.8,0.8,pert_redshift]
    lensModel = LensModel(lens_model_list=lens_model_list,z_source=z_source, lens_redshift_list=redshift_list, multi_plane=True)
    kwargs_nie = {'theta_E': theta_E, 'e1': e1, 'e2': e2, 's_scale': 1, 'center_x': 0, 'center_y': 0}
    kwargs_shear = {'gamma1': ext_shear_g1, 'gamma2': ext_shear_g2}
    kwargs_sis = {'theta_E': theta_E_sis, 'center_x': centre_x_sis, 'center_y': centre_y_sis}
    
    kwargs_lens = [kwargs_nie, kwargs_shear, kwargs_sis]
    
    #the light model
    kwargs_light_source = [{'amp': amp_source*100, 'R_sersic': 0.1, 'n_sersic': 1.5, 'center_x': beta_r, 'center_y': beta_d}]
    
    kwargs_light_nie = {'amp': amp_nie*theta_E*150, 'e1': e1, 'e2': e2,'s_scale': 1, 'center_x': 0, 'center_y': 0}
    kwargs_light_sis = {'amp': theta_E_sis*100, 'R_sersic': 0.05, 'n_sersic': 1, 'center_x': centre_x_sis, 'center_y': centre_y_sis}
    
    kwargs_light_lens = [kwargs_light_nie,kwargs_light_sis]
    
    #the point source model
    pointSource = PointSource(point_source_type_list=point_source_model_list, lensModel=lensModel, fixed_magnification_list=[True])
    kwargs_ps = [{'ra_source': beta_r, 'dec_source': beta_d, 'source_amp': 1}]

    #the image model

    imageModel = ImageModel(data_class=pixel_grid, psf_class=psf, lens_model_class=lensModel,
                        source_model_class=lightModel_source,
                        lens_light_model_class=lightModel_lens,
                        point_source_class=None, # in this example, we do not simulate point source.
                        kwargs_numerics=kwargs_numerics)

    # simulate image with the parameters we have defined above #

    image = imageModel.image(kwargs_lens=kwargs_lens, kwargs_source=kwargs_light_source,
                         kwargs_lens_light=kwargs_light_lens, kwargs_ps=kwargs_ps)


    # we can also add noise #

    exp_time = 100  # exposure time to quantify the Poisson noise level

    background_rms = 0.1  # background rms value

    poisson = image_util.add_poisson(image, exp_time=exp_time)

    bkg = image_util.add_background(image, sigma_bkd=background_rms)

    image_noisy = image + bkg + poisson

    if PlotImages:
        f, axes = plt.subplots(1, 2, figsize=(8, 4), sharex=False, sharey=False)

        axes[0].matshow(np.log10(image), origin='lower',cmap='hot')

        axes[1].matshow(np.log10(image_noisy), origin='lower',cmap='Blues')

        f.tight_layout()
    
    #the lens mapping plot
    if PlotLensMap:
        f, axes = plt.subplots(1, 1, figsize=(6, 6), sharex=False, sharey=False)
        lens_plot.lens_model_plot(axes, lensModel=lensModel, kwargs_lens=kwargs_lens, sourcePos_x=beta_r, sourcePos_y=beta_d, point_source=True, with_caustics=True, fast_caustic=True, coord_inverse=False)

    
interact(function,theta_E=2,theta_E_sis=(0,1,0.1), e1=(-0.5,0.5,0.2), e2=(-0.5,0.5,0.2), ext_shear_g1=(-0.1,0.1,0.02), ext_shear_g2=(-0.1,0.1,0.02),centre_x_sis=(-2,2,0.05),centre_y_sis=(-2,2,0.05),pert_redshift=(0.1,1.4,0.1),amp_source = (0,10,1),amp_nie = (0,10,1), beta_r=(-2,2,0.05),beta_d=(-2,2,0.05),PlotImages=True,PlotLensMap=False)

interactive(children=(IntSlider(value=2, description='theta_E', max=6, min=-2), FloatSlider(value=-0.099999999…

<function __main__.function(theta_E, e1, e2, ext_shear_g1, ext_shear_g2, theta_E_sis, centre_x_sis, centre_y_sis, pert_redshift, amp_source, amp_nie, beta_r, beta_d, PlotImages, PlotLensMap)>